In [1]:
import os
import sys
import torch
import numpy as np
import pandas as pd
import argparse
import torchvision.transforms as T
from data_utils import *

from torch.utils.data import DataLoader, Dataset
import pickle
from PIL import Image
os.chdir("..")
from src.utils import *

In [4]:
for subject in tqdm(range(1,9)):
    create_whole_region_imagery_unnormalized(subject, mask=True)
    create_whole_region_imagery_normalized(subject, mask=True)

  0%|          | 0/8 [00:00<?, ?it/s]

FileNotFoundError: No such file or no access: 'data/nsddata/ppdata/subj01/func1pt8mm/roi/nsdgeneral.nii.gz'

In [ ]:
def zscore(x, mean=None, stddev=None, return_stats=False):
    new_x = x
    if mean is not None:
        m = mean
    else:
        m = torch.mean(new_x, axis=0, keepdims=True)
    if stddev is not None:
        s = stddev
    else:
        s = torch.std(new_x, axis=0, keepdims=True)
    if return_stats:
        return (x - m)/(s+1e-6), m, s
    else:
        x = torch.where(s==0, (new_x - m), (new_x - m)/s)
        return x

In [ ]:
# nsd_root = '/export/raid1/home/surly/raid1/kendrick-data/nsd/'
# stim_dir = nsd_root + 'nsddata_stimuli/stimuli/nsd/'
# beta_dir = nsd_root + 'nsddata_betas/ppdata/'
# mask_dir= nsd_root + 'nsddata/ppdata/'
# img_stim_file = stim_dir + "nsdimagery_stimuli.pkl3"
# beta_subj = beta_dir + "subj%02d/func1pt8mm/nsdimagerybetas_fithrf/betas_nsdimagery.nii.gz"%subj

def image_feature_fn(image):
    '''take uint8 image and return floating point (0,1), either color or bw'''
    return image.astype(np.float32) / 255

## LOAD THE STIM IMAGES AND SEQUENCE ALIGNMENT DESCRIPTORS
stim_dir = "data/nsddata_stimuli/stimuli/nsd/"
img_stim_file = stim_dir + "nsdimagery_stimuli.pkl3"
ex_file = open(img_stim_file, 'rb')
imagery_dict = pickle.load(ex_file)
print(imagery_dict.keys())
ex_file.close()
exps = imagery_dict['exps']
cues = imagery_dict['cues']
image_map  = imagery_dict['image_map']
image_data = imagery_dict['image_data']
# for i, im in enumerate(image_data):
#     image = Image.fromarray(im.transpose(1,2,0))
#     image.save("data/nsddata_stimuli/stimuli/imagery_images/{}.png".format(i))

dict_keys(['meta_dm', 'gdm', 'gcm', 'image_data', 'image_map', 'exps', 'cues', 'modes'])


In [ ]:
def zscore(x, mean=None, stddev=None, return_stats=False):
    if mean is not None:
        m = mean
    else:
        m = np.mean(x, axis=0, keepdims=True)
    if stddev is not None:
        s = stddev
    else:
        s = np.std(x, axis=0, keepdims=True)
    if return_stats:
        return (x - m)/(s+1e-6), m, s
    else:
        return (x - m)/(s+1e-6)


## EXAMPLE CONDITION AVERAGED RESPONSES
def condition_average(data, cond):
    idx, idx_count = np.unique(cond, return_counts=True)
    idx_list = [cond==i for i in np.sort(idx)]
    avg_data = np.zeros(shape=(len(idx),)+data.shape[1:], dtype=np.float32)
    for i,m in enumerate(idx_list):
        avg_data[i] = np.mean(data[m], axis=0)
    return avg_data

meta_cond_idx = {
    'visA': np.arange(len(exps))[exps=='visA'],
    'visB': np.arange(len(exps))[exps=='visB'],
    'imgA_1': np.arange(len(exps))[exps=='imgA_1'],
    'imgA_2': np.arange(len(exps))[exps=='imgA_2'],
    'imgB_1': np.arange(len(exps))[exps=='imgB_1'],
    'imgB_2': np.arange(len(exps))[exps=='imgB_2']
}

cond_idx = {
    'visA': np.arange(len(exps))[exps=='visA'],
    'visB': np.arange(len(exps))[exps=='visB'],
    'imgA': np.arange(len(exps))[np.logical_or(exps=='imgA_1', exps=='imgA_2')],
    'imgB': np.arange(len(exps))[np.logical_or(exps=='imgB_1', exps=='imgB_2')]
}


In [ ]:
## LOAD THE DATA WITHOUT GLOBAL ZSCORING (EVEN IF IT IS ZSCORED WE ARE GONNA RE-ZSCORE LATER)

# voxel_data_raw = {}
# subjects = [1,2,3,4,5,6,7,8]
# for k,s in enumerate(subjects):
#     # print ('--------  subject %d  -------' % s)
#     create_whole_region_imagery_unnormalized(s)
#     create_whole_region_imagery_normalized(s)
    # voxel_data_raw[s] = torch.load("data/preprocessed_data/subject{}/nsd_imagery_unnormalized.pt".format(s))
    # print (voxel_data_raw[s].shape)


# ## NORMALIZATION OF THE DATA FOR EACH INDIVIDUAL TRIAL
# voxel_data_n = {}
# for s in voxel_data_raw.keys():
#     voxel_data_n[s] = np.copy(voxel_data_raw[s])
#     for c,idx in meta_cond_idx.items():
#         voxel_data_n[s][idx] = zscore(voxel_data_raw[s][idx])
#     print(voxel_data_n[s])


# cond_im_idx = {n: [image_map[c] for c in cues[idx]] for n,idx in cond_idx.items()}
# ## EXAMPLE USE
# for c, idx, im_idx in zip_dict(cond_idx, cond_im_idx): # loop conditions
#     data_single = voxel_data_n[s][idx]
#     data = condition_average(data_single, im_idx)
#     print(data.shape)

In [2]:
x, y = load_nsd_mental_imagery(vector = "images", subject=1, mode="attention", stimtype="concepts", average=True, nest=True, epoch=1)
# x, y = load_nsd_mental_imagery(vector = "images", subject=1, mode="vision", stimtype="concepts", average=False, nest=True)
# x, y = load_nsd_mental_imagery(vector = "c", subject=1, mode="imagery", stimtype="all", average=False, nest=True)
# x, y = load_nsd_mental_imagery(vector = "c", subject=1, mode="vision", stimtype="all", average=False, nest=True) 


['H', 'H', 'L', 'L', 'H', 'H', 'P', 'P', 'P', 'P', 'L', 'L', 'H', 'H', 'R', 'R', 'P', 'P', 'P', 'P', 'V', 'V', 'P', 'P', 'L', 'L', 'H', 'H', 'L', 'L', 'H', 'H', 'L', 'L', 'P', 'P', 'R', 'R', 'V', 'V', 'L', 'L', 'V', 'V', 'L', 'L', 'R', 'R', 'P', 'P', 'V', 'V', 'R', 'R', 'P', 'P', 'H', 'H', 'V', 'V', 'R', 'R', 'P', 'P', 'P', 'P', 'H', 'H', 'V', 'V', 'P', 'P', 'R', 'R', 'L', 'L', 'P', 'P', 'V', 'V', 'P', 'P', 'R', 'R', 'P', 'P', 'R', 'R', 'P', 'P', 'V', 'V', 'P', 'P', 'H', 'H']
torch.Size([6, 8, 15724])


In [ ]:
vector = "c"
subject = 1
mode = "imagery"
# stimtype="all"
stimtype="concepts"
average=False
nest=True

img_stim_file = "data/nsddata_stimuli/stimuli/nsd/nsdimagery_stimuli.pkl3"
ex_file = open(img_stim_file, 'rb')
imagery_dict = pickle.load(ex_file)
ex_file.close()
exps = imagery_dict['exps']
cues = imagery_dict['cues']
image_map  = imagery_dict['image_map']
image_data = imagery_dict['image_data']
print(image_map)

cond_idx = {
'visionsimple': np.arange(len(exps))[exps=='visA'],
'visioncomplex': np.arange(len(exps))[exps=='visB'],
'visionconcepts': np.arange(len(exps))[exps=='visC'],
'visionall': np.arange(len(exps))[np.logical_or(exps=='visA', exps=='visB')],
'imagerysimple': np.arange(len(exps))[np.logical_or(exps=='imgA_1', exps=='imgA_2')],
'imagerycomplex': np.arange(len(exps))[np.logical_or(exps=='imgB_1', exps=='imgB_2')],
'imageryconcepts': np.arange(len(exps))[np.logical_or(exps=='imgC_1', exps=='imgC_2')],
'imageryall': np.arange(len(exps))[np.logical_or(np.logical_or(exps=='imgA_1', exps=='imgA_2'), np.logical_or(exps=='imgB_1', exps=='imgB_2'))]
}
x = torch.load("data/preprocessed_data/subject{}/nsd_imagery.pt".format(subject)).requires_grad_(False).to("cpu")
if stimtype != "concepts":
    cond_im_idx = {n: [image_map[c] for c in cues[idx]] for n,idx in cond_idx.items()}
    y = torch.load("data/preprocessed_data/{}_18.pt".format(vector)).requires_grad_(False).to("cpu")
else:
    cond_im_idx = {n: list(cues[idx]) for n,idx in cond_idx.items()}
    concept_map = {"S" : "stripes", "Z" :"zebra", "M" : "mammal", "Y" : "yellow", "N" : "banana", "F" : "fruit"}
    y = ["stripes", "zebra", "mammal", "yellow", "banana", "fruit"]

# Prune down to specific experimental mode/stimuli type
x = x[cond_idx[mode+stimtype]]
averaged_x, sample_count = condition_average(x, cond_im_idx[mode+stimtype])
# Letter cues get sorted out of order, so we need to fix the order
trial_count = int(x.shape[0]/sample_count)
# Average across trials
if average:
    x =  averaged_x
    if stimtype == "concepts":
        order = [3, 5, 1, 4, 2, 0]
        x = x[order]
    x = x.reshape((x.shape[0], 1, x.shape[1]))
    
elif nest:
    x_new = torch.zeros((sample_count, trial_count, x.shape[1]))
    for i in range(sample_count):
        x_new[i] = x[i*trial_count: i*trial_count + trial_count]
    x = x_new


if stimtype == "simple":
    y = y[:6]
elif stimtype == "complex":
    y = y[6:12]
elif stimtype == "concepts":
    y = y[12:]

{'H': 0, 'R': 1, 'V': 2, 'L': 3, 'P': 4, 'E': 5, 'W': 6, 'K': 7, 'B': 8, 'C': 9, 'D': 10, 'T': 11, 'S': 12, 'Z': 13, 'M': 14, 'Y': 15, 'N': 16, 'F': 17}
torch.Size([6, 16, 15724]) ['stripes', 'zebra', 'mammal', 'yellow', 'banana', 'fruit']


In [ ]:
def load_nsd_attention(vector = "images", subject=1, mode="attention", stimtype="all", average=False, nest=True):
    img_stim_file = "data/nsddata_stimuli/stimuli/nsd/nsdimagery_stimuli.pkl3"
    imagery_data_path = "/export/raid1/home/tsaharoy/NSD_Imagery/everything_NSD_imagery"
    ex_file = open(img_stim_file, 'rb')
    imagery_dict = pickle.load(ex_file)
    ex_file.close()
    exps = imagery_dict['exps']
    cues = imagery_dict['cues']
    image_map  = imagery_dict['image_map']
    image_data = imagery_dict['image_data']
    cond_idx = {
    'visionsimple': np.arange(len(exps))[exps=='visA'],
    'visioncomplex': np.arange(len(exps))[exps=='visB'],
    'visionconcepts': np.arange(len(exps))[exps=='visC'],
    'visionall': np.arange(len(exps))[np.logical_or(np.logical_or(exps=='visA', exps=='visB'), exps=='visC')],
    'imagerysimple': np.arange(len(exps))[np.logical_or(exps=='imgA_1', exps=='imgA_2')],
    'imagerycomplex': np.arange(len(exps))[np.logical_or(exps=='imgB_1', exps=='imgB_2')],
    'imageryconcepts': np.arange(len(exps))[np.logical_or(exps=='imgC_1', exps=='imgC_2')],
    'imageryall': np.arange(len(exps))[np.logical_or(
                                        np.logical_or(
                                            np.logical_or(exps=='imgA_1', exps=='imgA_2'), 
                                            np.logical_or(exps=='imgB_1', exps=='imgB_2')), 
                                        np.logical_or(exps=='imgC_1', exps=='imgC_2'))],
    'attentionsimple': np.arange(len(exps))[exps=='attA'],
    'attentioncomplex': np.arange(len(exps))[exps=='attB'],
    'attentionconcept': np.arange(len(exps))[exps=='attC'],
    'attentionall': np.arange(len(exps))[np.logical_or(
                                            np.logical_or(exps=='attA', exps=='attB'), 
                                            exps=='attC')]}
    
    x = torch.load("data/preprocessed_data/subject{}/nsd_imagery.pt".format(subject)).requires_grad_(False).to("cpu")
    cond_im_idx = {n: [image_map[c] for c in cues[idx]] for n,idx in cond_idx.items()}
    y = torch.load("data/preprocessed_data/{}_18.pt".format(vector)).requires_grad_(False).to("cpu")
    x = x[cond_idx[mode+stimtype]]
    # Prune down to specific experimental mode/stimuli type
    if stimtype == "simple":
        identifiers = ["attA"]
    elif stimtype == "complexs":
        identifiers = ["attB"]
    elif stimtype == "concepts":
        identifiers = ['attC']
    else:
        identifiers = ["attA", "attB", "attC"]
    dfs = []
    for identifier in identifiers:
        task_framefile = f"{imagery_data_path}/experiment/runs/{identifier}/{identifier}_framefile.csv"
        assert os.path.exists(task_framefile)
        df = pd.read_csv(task_framefile)
        dfs.append(df)
    concatenated_df = pd.concat(dfs)
    run_data = []
    current_cue = None
    current_trial = {"trial": None, "cue" : None, "stimulus" : None, "barrage" : [], "stim_present" : None, "voxels" : None}
    trial = -1
    # iterate through frames of the experiment to collect the data presented in each trial
    for index, row in concatenated_df.iterrows():
        cue = row[1]
        letter = cue.split("cue")[1][0]
        # check if this frame is a new cue frame
        if letter != current_cue and letter != "X" and "blank" in row[1]:
            # if it is a cue frame and not our first, add the previous trial to the data structure
            if trial != -1:
                print(index, cues[cond_idx[mode+stimtype][trial*2:trial*2+2]], current_trial)
                run_data.append(current_trial)
            trial +=1
            stim_img = Image.open(f"data/nsddata_stimuli/stimuli/imagery_images/{image_map[letter]}.png")
            current_trial = {"trial": trial, "cue" : letter, "stimulus" : stim_img, "barrage" : [], "barrage_stim_only" : [], "stim_present" : row[0] != 0, "positive_stim" : row[0] == 2, "voxels" : x[trial*2:trial*2+2]}
            current_cue = letter
        # If it is a frame with a stimulus, add it to the barrage data
        elif letter == "X" and "blank" not in row[1]:
            img_identifier = row[1].split("cue")[0][5:-1] + ".png"
            image = search_and_open_image(f"{imagery_data_path}/images/raw_images/", img_identifier)
            current_trial["barrage"].append(image)
            if row[0] == 2:
                current_trial["barrage_stim_only"].append(image)
        # If it is a cue frame that we have already seen, skip it
        elif letter == current_cue and "blank" in row[1]:
            pass
    run_data.append(current_trial)
    # cues_list = [data["cue"] for data in run_data]
    # print(np.unique(cues_list, return_counts=True))
    # print(cues_list)
    # print([cues[idx] for idx in cond_idx[mode+stimtype]])

    if stimtype == "simple":
        y = y[:6]
    elif stimtype == "complex":
        y = y[6:12]
    elif stimtype == "concepts":
        y = y[12:]
        
        
    
    
load_nsd_attention(stimtype="simple")

192 ['V' 'V'] {'trial': 0, 'cue': 'V', 'stimulus': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=227x227 at 0x7F277F2CA290>, 'barrage': [<PIL.PngImagePlugin.PngImageFile image mode=L size=714x714 at 0x7F277F2CA200>, <PIL.PngImagePlugin.PngImageFile image mode=L size=714x714 at 0x7F277F2C9F90>, None, None, <PIL.PngImagePlugin.PngImageFile image mode=L size=714x714 at 0x7F277F2CBC40>, <PIL.PngImagePlugin.PngImageFile image mode=L size=714x714 at 0x7F277F2CBCD0>, <PIL.PngImagePlugin.PngImageFile image mode=L size=714x714 at 0x7F277F2CBC70>, <PIL.PngImagePlugin.PngImageFile image mode=L size=714x714 at 0x7F277F2CBC10>, <PIL.PngImagePlugin.PngImageFile image mode=L size=714x714 at 0x7F277F2CA050>, <PIL.PngImagePlugin.PngImageFile image mode=L size=714x714 at 0x7F277F2CBEE0>, <PIL.PngImagePlugin.PngImageFile image mode=L size=714x714 at 0x7F277F2CB730>, <PIL.PngImagePlugin.PngImageFile image mode=L size=714x714 at 0x7F277F2CB580>, None, <PIL.PngImagePlugin.PngImageFile image mode=L si

In [ ]:
y = format_imagery_stimuli()
torch.save(y, "data/preprocessed_data/images_18.pt")

In [ ]:
import scipy.io

mat_file = '/home/naxos2-raid25/kneel027/home/kneel027/home/tsaharoy/NSD_Imagery/everything_NSD_imagery/behavioral/button_presses/imageryscan_output_files/20191105-NSD258-nsdimagery/20191105161734_nsdimagery_subj258_run01_exp01.mat'

mat_data = scipy.io.loadmat(mat_file)
print(mat_data)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Tue Nov  5 16:25:40 2019', '__version__': '1.0', '__globals__': [], 'C': array([[array([[array(['n/a'], dtype='<U3')],
               [array(['n/a'], dtype='<U3')],
               [array(['n/a'], dtype='<U3')],
               ...,
               [array(['n/a'], dtype='<U3')],
               [array(['n/a'], dtype='<U3')],
               [array(['n/a'], dtype='<U3')]], dtype=object),
        array([[array(['stim/blank00000_00000000_cueX.png'], dtype='<U33')],
               [array(['stim/blank00000_00000000_cueX.png'], dtype='<U33')],
               [array(['stim/blank00000_00000000_cueX.png'], dtype='<U33')],
               ...,
               [array(['stim/blank00000_00000000_cueX.png'], dtype='<U33')],
               [array(['stim/blank00000_00000000_cueX.png'], dtype='<U33')],
               [array(['stim/blank00000_00000000_cueX.png'], dtype='<U33')]],
              dtype=object)                                     

In [8]:
for subject in [1,2,5,7]:
    x, y = load_nsd_mental_imagery(vector = "images", subject=subject, mode="vision", stimtype="all", average=True, nest=False, epoch=0)
    print(x.shape, y.shape)
    np.save(f"/home/naxos2-raid25/kneel027/home/kneel027/brain-diffuser_mi/data/processed_data/subj0{subject}/nsd_test_fmriavg_nsdgeneral_sub{subject}_vis.npy", x.numpy())
    np.save(f"/home/naxos2-raid25/kneel027/home/kneel027/brain-diffuser_mi/data/processed_data/subj0{subject}/nsd_test_stim_sub{subject}_mi.npy", y.reshape(18, 425, 425, 3).numpy())
for subject in [1,2,5,7]:
    x, y = load_nsd_mental_imagery(vector = "images", subject=subject, mode="imagery", stimtype="all", average=True, nest=False, epoch=0)
    print(x.shape)
    np.save(f"/home/naxos2-raid25/kneel027/home/kneel027/brain-diffuser_mi/data/processed_data/subj0{subject}/nsd_test_fmriavg_nsdgeneral_sub{subject}_img.npy", x.numpy())

torch.Size([18, 1, 15724])
torch.Size([18, 1, 15724]) torch.Size([18, 541875])
torch.Size([18, 1, 14278])
torch.Size([18, 1, 14278]) torch.Size([18, 541875])
torch.Size([18, 1, 13039])
torch.Size([18, 1, 13039]) torch.Size([18, 541875])
torch.Size([18, 1, 12682])
torch.Size([18, 1, 12682]) torch.Size([18, 541875])
torch.Size([18, 1, 15724])
torch.Size([18, 1, 15724])
torch.Size([18, 1, 14278])
torch.Size([18, 1, 14278])
torch.Size([18, 1, 13039])
torch.Size([18, 1, 13039])
torch.Size([18, 1, 12682])
torch.Size([18, 1, 12682])
